In [18]:

import pandas_datareader as web
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpl
import numpy as np
import requests
import datetime as dt
import os
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model



def main():
    model_stock = 'sandp500'
    stock = 'sandp500'
    model = load_model('../../machine_learning/best_model/model/{}/model_2.h5'.format(model_stock))
    
    
    df = pd.read_csv('../../stock_data/stock_code/{}.csv'.format(stock))
    print(df)

    signs = []
    for symbol in df['symbol']:
        signs.append(symbol)


    # content = '\n\nToday`s stock report' + str(dt.date.today())
    count = 0
    position = pd.DataFrame()

    # #----------------------------------------------------------------------------
    for i in signs :   
        start=dt.datetime.now()-dt.timedelta(days=400)
        end=dt.datetime.now()
        count += 1
        
        # print(start)
        # print(count)
        print(i)
        try:
            df = yf.download(i, start, end, interval='1d')
            # print(df)

            feature1(df)
            print(df)

            cols = ['sma30','sma60', 'sma90','sma120' ,'sma150','sma180','sma210','sma240','sma270'

                    ,'sma75_30', 'sma75_90', 'sma75_150', 'sma75_210', 'sma105_30', 'sma105_90',
                    'sma105_150', 'sma105_210', 'sma135_30', 'sma135_90', 'sma135_150', 'sma135_210'

                ,'Highest81','Highest81,30days_ago','Highest81,60days_ago','Highest121','Highest121,30days_ago'
                    ,'Highest121,60days_ago','Highest161','Standard_deviation_normalization30',
                    'Standard_deviation_normalization35','Standard_deviation_normalization40'
                ,'adosc']
            std_model = StandardScaler()
            df[cols] = std_model.fit_transform(df[cols])
#             print(df[cols])
        
            df[i] = model.predict(df[cols])
            print(df[i])

            position = pd.concat([position ,df[i]],axis = 1)
            print(position)

        except:
            continue
    
    print(position)
    series = position.iloc[-1]
    print(series)
    series = series.sort_values(ascending=False) 
    code_list = series.index.values.tolist()
    point_list = series.values.tolist()
#             どのくらい抽出するのか
    pickup = 5
    code_list = code_list[0:pickup]
    point_list = point_list[0:pickup]
    print(code_list[0:pickup])
    print(point_list[0:pickup])

    content = [code_list,point_list]
#     for i,j in code_list,point_list:
#         content.append('')
    send_line_notify(content)





            
        



def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'GlXSB9BURpdoAifXsABCBhAyNoehXTZ6Gp7Z2E8Wc2I'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)
    

def feature1(df):

    
    # simple moving average
    for i in range(30,300,30):
        df['sma'+str(i)] = df['Adj Close'].rolling(i).mean()
        df['sma'+str(i)] = df['sma'+str(i)] / df['sma'+str(i)].shift(1)


    for i in range(15,150,30):
        for k in range(30,270,60): 
            df['sma'+str(k)] = df['Adj Close'].rolling(k).mean()
            df['sma'+str(i)] = df['Adj Close'].rolling(i).mean()
            df['sma'+str(i)+'_'+str(k)] = df['sma'+str(i)] / df['sma'+str(k)]

    for term in range(5,50,5):
        df['SMA'+str(term)] = df['Adj Close'].rolling(term).mean()
        df['STD'+str(term)] = df['Adj Close'].rolling(term).std()
        df['Standard_deviation_normalization'+str(term)] = 100 * 2 * df['STD'+str(term)] / df['SMA'+str(term)]

    for i in range(41,201,40):
        df['Highest'+str(i)] = df['Adj Close'].rolling(window=81).max()
        df['Highest'+str(i)] = df['Highest'+str(i)].shift()
        for m in range(30,90,30):
            df['Highest'+str(i)+','+str(m)+'days_ago'] = df['Adj Close'] / df['Highest'+str(i)].shift(m)

        #今日の終値が過去何日間の高音に対してどの程度あるか
        df['Highest'+str(i)] = df['Adj Close'] / df['Highest'+str(i)]


    # ADOSC
    df['adosc'] = ((2 * df['Close'] - df['High'] - df['Low']) / (df['High'] - df['Low'])) * df['Volume']
    df['adosc'] = df['adosc'].cumsum()

    df.dropna(inplace=True)

    return df

def Normalize(df):
    df_normalized = (df - df.mean(axis=0)) / df.std(axis=0)

def One_One_Scale(df):
    df_scaled = 2 * (df - df.min()) / (df.max() - df.min()) - 1
    return df_scaled
        
main()
    

    symbol                 Name                  Sector
0      MMM           3M Company             Industrials
1      AOS      A.O. Smith Corp             Industrials
2      ABT  Abbott Laboratories             Health Care
3     ABBV          AbbVie Inc.             Health Care
4     ABMD              Abiomed             Health Care
..     ...                  ...                     ...
500    YUM      Yum! Brands Inc  Consumer Discretionary
501   ZBRA   Zebra Technologies  Information Technology
502    ZBH        Zimmer Biomet             Health Care
503   ZION        Zions Bancorp              Financials
504    ZTS               Zoetis             Health Care

[505 rows x 3 columns]
MMM
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-04-12  198.199997  199.289993  197.059998  197.830002  197.830002   
2021-04-1

Date
2021-04-12    0.979795
2021-04-13    0.912464
2021-04-14    0.881617
2021-04-15    0.910268
2021-04-16    0.916953
Name: ABT, dtype: float32
                          MMM       AOS       ABT
2021-04-12 00:00:00  0.960208  0.966256  0.979795
2021-04-13 00:00:00  0.896806  0.904349  0.912464
2021-04-14 00:00:00  0.899543  0.928137  0.881617
2021-04-15 00:00:00  0.952674  0.908387  0.910268
2021-04-16 00:00:00  0.919400  1.009548  0.916953
ABBV
[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-04-12  107.820000  108.570000  107.360001  108.250000  106.949646   
2021-04-13  108.029999  108.360001  107.029999  108.220001  106.920006   
2021-04-14  106.220001  106.669998  105.580002  105.900002  105.900002   
2021-04-15  105.989998  106.949997  105.620003  106.889999  106.889999   
2021-04-16  107.489998  108.550003  

Date
2021-04-12    1.005457
2021-04-13    0.921775
2021-04-14    0.874902
2021-04-15    0.956419
2021-04-16    0.912517
Name: ACN, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN  
2021-04-12 00:00:00  1.005457  
2021-04-13 00:00:00  0.921775  
2021-04-14 00:00:00  0.874902  
2021-04-15 00:00:00  0.956419  
2021-04-16 00:00:00  0.912517  
ATVI
[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume  \
Date                                                                   

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2021-04-12  184.050003  188.529999  184.050003  188.059998  188.059998   
2021-04-13  188.770004  188.910004  185.440002  187.410004  187.410004   
2021-04-14  187.470001  189.490005  187.100006  187.770004  187.770004   
2021-04-15  188.289993  190.199997  188.089996  189.460007  189.460007   
2021-04-16  191.279999  193.169998  190.550003  192.919998  192.919998   

            Volume       sma30     sma60       sma90    sma120  ...  \
Date                                                            ...   
2021-04-12  613600  179.290517  1.001628  166.152280  1.001723  ...   
2021-04-13  632300  180.122967  1.001849  166.543901  1.001776  ...   
2021-04-14  765900  180.894850  1.002232  166.985609  1.001803  ...   
2021-04-15  609300  181.690111  1.002326  167.427372  1.0

Date
2021-04-12    1.041223
2021-04-13    0.917471
2021-04-14    0.908049
2021-04-15    0.934974
2021-04-16    0.943685
Name: AES, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626   
2021-04-16 00:00:00  0.912517  0.920869  0.940157 

...

Date
2021-04-12    0.919370
2021-04-13    0.896332
2021-04-14    0.871038
2021-04-15    0.887307
2021-04-16    0.925011
Name: WYNN, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  ...  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491  ...   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037  ...   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783  ...   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626  ...   
2021-04-16 00:00:00  0.9

Date
2021-04-12    0.915280
2021-04-13    0.907908
2021-04-14    0.882261
2021-04-15    0.963360
2021-04-16    0.928027
Name: XLNX, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  ...  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491  ...   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037  ...   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783  ...   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626  ...   
2021-04-16 00:00:00  0.9

Date
2021-04-12    0.991037
2021-04-13    0.935046
2021-04-14    0.898497
2021-04-15    0.914241
2021-04-16    0.967620
Name: YUM, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  ...  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491  ...   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037  ...   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783  ...   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626  ...   
2021-04-16 00:00:00  0.91

Date
2021-04-12    1.007718
2021-04-13    0.942417
2021-04-14    0.920567
2021-04-15    0.940883
2021-04-16    0.949303
Name: ZBH, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  ...  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491  ...   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037  ...   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783  ...   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626  ...   
2021-04-16 00:00:00  0.91

Date
2021-04-12    0.980413
2021-04-13    0.911319
2021-04-14    0.871036
2021-04-15    0.893802
2021-04-16    0.932324
Name: ZTS, dtype: float32
                          MMM       AOS       ABT      ABBV      ABMD  \
2021-04-12 00:00:00  0.960208  0.966256  0.979795  0.965514  1.017524   
2021-04-13 00:00:00  0.896806  0.904349  0.912464  0.914844  0.920174   
2021-04-14 00:00:00  0.899543  0.928137  0.881617  0.894541  0.878925   
2021-04-15 00:00:00  0.952674  0.908387  0.910268  0.917600  0.939098   
2021-04-16 00:00:00  0.919400  1.009548  0.916953  0.903428  0.961436   

                          ACN      ATVI      ADBE       AAP       AMD  ...  \
2021-04-12 00:00:00  1.005457  0.948111  0.987315  0.954093  0.925491  ...   
2021-04-13 00:00:00  0.921775  0.912873  0.914567  0.911588  0.904037  ...   
2021-04-14 00:00:00  0.874902  0.886082  0.888089  0.899969  0.875783  ...   
2021-04-15 00:00:00  0.956419  0.945234  0.934895  0.906429  0.965626  ...   
2021-04-16 00:00:00  0.91